In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

## 1. Data Generation (simulating climate change data)


In [3]:
def generate_climate_data(n_samples=1000):
    np.random.seed(42)
    
    # Features: deforestation_rate, industrial_emissions, solar_activity, time
    deforestation_rate = np.random.uniform(0, 1, n_samples)
    industrial_emissions = np.random.uniform(0, 100, n_samples)
    solar_activity = np.random.normal(0, 1, n_samples)
    time = np.linspace(0, 10, n_samples)
    
    # Target: temperature_change (simplified model)
    temperature_change = (
        0.3 * deforestation_rate +
        0.5 * industrial_emissions +
        0.1 * solar_activity +
        0.1 * time +
        np.random.normal(0, 0.1, n_samples)
    )
    temperature_change = (temperature_change - np.min(temperature_change)) / (np.max(temperature_change) - np.min(temperature_change))
    
    data = pd.DataFrame({
        'deforestation_rate': deforestation_rate,
        'industrial_emissions': industrial_emissions,
        'solar_activity': solar_activity,
        'time': time,
        'temperature_change': temperature_change
    })
    
    return data


## 2. Data Preprocessing


In [4]:
def preprocess_data(data):
    X = data[['deforestation_rate', 'industrial_emissions', 'solar_activity', 'time']]
    y = data['temperature_change']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test, scaler

## 3. Model Creation


In [5]:
def create_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

## 4. Model Training


In [6]:
def train_model(model, X_train, y_train):
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
    return history

## 5. Question Generation


In [7]:
def generate_question(model, scaler, question_type='multiple_choice'):
    # Generate random input
    input_data = np.random.rand(1, 4)
    scaled_input = scaler.transform(input_data)
    
    # Predict temperature change
    predicted_change = model.predict(scaled_input)[0][0]
    
    # Create question based on the prediction
    if question_type == 'multiple_choice':
        question = "Given the following conditions:\n"
        question += f"Deforestation Rate: {input_data[0][0]:.2f}\n"
        question += f"Industrial Emissions: {input_data[0][1]:.2f}\n"
        question += f"Solar Activity: {input_data[0][2]:.2f}\n"
        question += f"Time Period: {input_data[0][3]:.2f} decades\n\n"
        question += "What is the most likely impact on global temperature?\n"
        
        options = [
            f"A) High temperature increase (>{predicted_change+0.2:.2f}°C)",
            f"B) Moderate temperature increase ({predicted_change-0.1:.2f}°C - {predicted_change+0.1:.2f}°C)",
            f"C) Low temperature increase (<{predicted_change-0.2:.2f}°C)",
            "D) No significant change in temperature"
        ]
        
        correct_answer = 'B'
        
        return question, options, correct_answer
    
    elif question_type == 'true_false':
        statement = "Given the environmental conditions, "
        if predicted_change > 0.5:
            statement += "the global temperature is likely to increase significantly."
            correct_answer = True
        else:
            statement += "the global temperature is likely to remain relatively stable."
            correct_answer = False
        
        return statement, correct_answer

## 6. Main Execution


In [ ]:
if __name__ == "__main__":
    # Generate and preprocess data
    data = generate_climate_data()
    X_train, X_test, y_train, y_test, scaler = preprocess_data(data)
    
    # Create and train model
    model = create_model((X_train.shape[1],))
    history = train_model(model, X_train, y_train)
    
    # Evaluate model
    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test MAE: {test_mae:.4f}")
    
    # Generate questions
    print("\nMultiple Choice Question:")
    mc_question, mc_options, mc_answer = generate_question(model, scaler, 'multiple_choice')
    print(mc_question)
    for option in mc_options:
        print(option)
    print(f"Correct Answer: {mc_answer}")
    
    print("\nTrue/False Question:")
    tf_statement, tf_answer = generate_question(model, scaler, 'true_false')
    print(tf_statement)
    print(f"Correct Answer: {tf_answer}")

## 7. Interactive Quiz Function


In [9]:
def take_quiz(model, scaler, num_questions=5):
    score = 0
    for i in range(num_questions):
        print(f"\nQuestion {i+1}:")
        if np.random.rand() > 0.5:
            # Multiple Choice Question
            question, options, correct_answer = generate_question(model, scaler, 'multiple_choice')
            print(question)
            for option in options:
                print(option)
            user_answer = input("Your answer (A/B/C/D): ").upper()
            if user_answer == correct_answer:
                print("Correct!")
                score += 1
            else:
                print(f"Incorrect. The correct answer is {correct_answer}.")
        else:
            # True/False Question
            statement, correct_answer = generate_question(model, scaler, 'true_false')
            print(statement)
            user_answer = input("True or False? (T/F): ").upper() == 'T'
            if user_answer == correct_answer:
                print("Correct!")
                score += 1
            else:
                print(f"Incorrect. The correct answer is {correct_answer}.")
    
    print(f"\nQuiz completed! Your score: {score}/{num_questions}")

## 8. Run the quiz


In [ ]:
if __name__ == "__main__":
    print("\nWelcome to the Climate Change Impact Quiz!")
    take_quiz(model, scaler)

## 9. Educational Information


In [11]:
def provide_climate_change_info():
    print("\nClimate Change Information:")
    print("1. Deforestation and Climate Change:")
    print("   - Deforestation contributes to climate change by releasing stored carbon into the atmosphere.")
    print("   - Forests act as carbon sinks, absorbing CO2 from the atmosphere.")
    print("   - When forests are cleared, this carbon storage capacity is lost.")
    print("\n2. Greenhouse Gas Emissions:")
    print("   - Industrial activities release greenhouse gases like CO2 into the atmosphere.")
    print("   - These gases trap heat, leading to global warming.")
    print("   - The concentration of CO2 in the atmosphere has increased significantly since the Industrial Revolution.")
    print("\n3. Solar Activity:")
    print("   - While solar activity can influence Earth's climate, its effect is much smaller compared to human activities.")
    print("   - Changes in solar radiation contribute to natural climate variability.")
    print("\n4. Time Factor:")
    print("   - Climate change effects accumulate over time.")
    print("   - Long-term trends are more significant than short-term fluctuations.")
    print("\n5. Interconnected Systems:")
    print("   - Climate change can lead to various environmental impacts, including:")
    print("     * Rising sea levels")
    print("     * More frequent and intense extreme weather events")
    print("     * Changes in precipitation patterns")
    print("     * Impacts on biodiversity and ecosystems")

## 10. Run the educational program


In [ ]:
if __name__ == "__main__":
    print("\nWelcome to the Climate Change Educational Program!")
    provide_climate_change_info()
    input("\nPress Enter to start the quiz...")
    take_quiz(model, scaler)

In [1]:
import joblib
from sklearn.ensemble import RandomForestClassifier

# Create a scikit-learn model
model = RandomForestClassifier()

# Train the model on your data

# Save the model to a file
joblib.dump(model, 'climate change.pkl')

['climate change.pkl']